In [ ]:
#Imports and libraries 
import numpy as np
import panda as pd

In [ ]:
#Loading Training Data
TrainingData = pd.read_csv('trg.csv')

In [ ]:
#Preprocessing


In [ ]:
#Cross-Validation Test
folds = 10
cycle = 1
performance = {}

#for(cycle=1; cycle<=folds; cycle++)
for x in DataTraining 
    testFold = DataTraining[cycle]
    trainingFolds = DataTraining.pop(cycle)
    performance[cycle] = runtest(trainingFolds, testFold)
    cycle++ 

averagePerformance = np.sum(performance)/folds
    

In [ ]:
#Generate Solution and Export to CSV

df.to_cvs('amon897.csv')